In [ ]:
# Install required packages
!pip install -q plotly pandas requests

# Imports
import pandas as pd
import requests
import plotly.express as px
from collections import Counter

# Step 1: Load ISO metadata
iso_url = "https://raw.githubusercontent.com/IFRCGo/monty-stac-extension/5854516465eb565b689bf8b643d0ed5401e53ccd/docs/model/Montandon_JSON-Example.json"
iso_response = requests.get(url=iso_url)
iso_data = iso_response.json()['taxonomies']['ISO_info']

# Build ISO3 → Country name mapping
iso3_to_country = {item['ISO3']: item['country'] for item in iso_data}

# Step 2: Collect STAC country codes
collection_list = ['ifrcevent-events', 'pdc-events'] # Note Add all collection id
country_list = []

for source_event in collection_list:
    start_date = "2020-01-01T00:00:00Z"
    end_date = "2026-01-01T00:00:00Z"
    datetime_range = f"{start_date}/{end_date}"

    url = (
        f"https://montandon-eoapi-stage.ifrc.org/stac/collections/"
        f"{source_event}/items?limit=200&datetime={datetime_range}"
    )

    while url:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data for {source_event}: {response.status_code}")
            break

        data = response.json()
        for item in data.get('features', []):
            codes = item['properties'].get('monty:country_codes', [])
            country_list.extend(codes)

        next_link = next((l.get("href") for l in data.get("links", []) if l.get("rel") == "next"), None)
        url = next_link

# Step 3: Count country code frequencies
country_event_counts = dict(Counter(country_list))

# Step 4: Prepare DataFrame for Plotly
df = pd.DataFrame([
    {"iso3": iso, "country": iso3_to_country.get(iso, iso), "event_count": count}
    for iso, count in country_event_counts.items()
])

# Step 5: Create Plotly Choropleth
fig = px.choropleth(
    df,
    locations="iso3",              # ISO3 codes
    color="event_count",           # Color by event count
    hover_name="country",          # Hover label
    color_continuous_scale="OrRd",
    title="Disaster Event Counts by Country (2020–2025)",
    labels={"event_count": "Event Count"},
)

fig.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

# Show the map
fig.show()

print(country_event_counts)

{'COG': 10, 'ECU': 112, 'COD': 97, 'SOM': 9, 'MMR': 125, 'TZA': 14, 'SLE': 5, 'KGZ': 6, 'MYS': 11, 'ARG': 44, 'CMR': 45, 'BWA': 3, 'ZWE': 5, 'UGA': 6, 'CHL': 21, 'GNB': 2, 'GHA': 2, 'AGO': 188, 'ETH': 9, 'VUT': 8, 'GAB': 9, 'HND': 16, 'COL': 63, 'CRI': 10, 'PAN': 14, 'PHL': 63, 'CUB': 7, 'DZA': 6, 'NPL': 24, 'TCD': 45, 'PRT': 75, 'ROU': 16, 'POL': 4, 'VNM': 66, 'PAK': 18, 'CPV': 2, 'BDI': 4, 'SSD': 4, 'BOL': 217, 'NGA': 9, 'BRB': 1, 'GRD': 4, 'JAM': 2, 'VCT': 5, 'LBR': 3, 'ZAF': 7, 'ARM': 2, 'PNG': 8, 'NAM': 2, 'BGD': 14, 'BLZ': 14, 'KEN': 12, 'GTM': 42, 'LSO': 2, 'BRA': 101, 'YEM': 8, 'AFG': 9, 'MOZ': 8, 'MDG': 4, 'KAZ': 8, 'URY': 2, 'BFA': 16, 'COM': 2, 'BIH': 38, 'MHL': 2, 'ZMB': 9, 'GIN': 6, 'MDV': 1, 'FSM': 1, 'DOM': 4, 'FJI': 7, 'MNG': 13, 'ISL': 28, 'MEX': 125, 'LKA': 7, 'LBY': 1, 'MLI': 10, 'MAR': 1, 'NER': 17, 'IDN': 92, 'MRT': 11, 'SYR': 6, 'GRC': 35, 'SRB': 13, 'ITA': 21, 'MWI': 3, 'SVN': 1, 'GNQ': 1, 'TUR': 12, 'IRN': 13, 'TTO': 1, 'CAF': 9, 'TGO': 1, 'LBN': 1, 'THA': 36, '

In [1]:
import os

# Number of CPU cores (physical + virtual)
print(os.cpu_count())


12
